In [1]:
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

## Context Offloading: Filesystem

<img src="./assets/agent_header_files.png" width="800" style="display:block; margin-left:0;">

Agent context windows can grow rapidly during complex tasks—the average Manus task uses approximately 50 tool calls, creating substantial context accumulation. A powerful technique for managing this growth is **context offloading** through filesystem operations. Rather than storing all tool call observations and intermediate results directly in the context window, agents can strategically save information to files and [fetch it as-needed](https://blog.langchain.com/context-engineering-for-agents/), maintaining focus while preserving access to critical information.

This approach has been successfully implemented by production systems like [Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus) and [Hugging Face Open Deep Research](https://huggingface.co/blog/open-deep-research). Anthropic's [multi-agent research system](https://www.anthropic.com/engineering/multi-agent-research-system#:~:text=Subagent%20output%20to%20a%20filesystem%20to%20minimize%20the%20%E2%80%98game%20of%20telephone.%E2%80%99) provides another compelling example, where subagents store their work in external systems and pass lightweight references back to coordinators. This prevents the "game of telephone" effect—information degradation as it passes through multiple agents—while enabling fresh subagents to spawn with clean contexts and retrieve stored context like research plans from memory when needed.

By writing token-heavy context to files in sandboxed environments, agents can effectively manage memory while maintaining the ability to retrieve detailed information when necessary. This pattern is particularly valuable for structured outputs like code, reports, or data visualizations where specialized prompts produce better results than filtering through general coordinators, and for long-running research tasks where intermediate results need preservation without constant attention.

### File Tools

Our implementation uses a virtual filesystem approach that mocks the traditional filesystem within LangGraph state. The core insight is to use a simple dictionary where keys represent mock file paths and values contain the file content. This approach provides short-term, thread-wise persistence that's ideal for maintaining context within a single agent conversation, though it's not suited for information that needs to persist across different conversation threads. The file operations utilize LangGraph's `Command` type to update the agent state, enabling tools to modify the virtual filesystem and maintain proper state management throughout the agent's execution.

You’ll build three file tools—`ls`, `read_file`, and `write_file`—that operate on the virtual file system.

**Usage:** 
- When the LLM has information in its context that it wants to persist, it writes it to a file with `write_file`; later, the same agent or a subagent can retrieve it with `read_file`.
- A tool call can write data to a file and provide the filename in the tool call return message to the LLM. The LLM can later decide to read some or all of the content, or could apply another tool to process the data. 
- Use `ls` to list available files.

The read/write tools expect newline-delimited plain text (parsed with `str.splitlines()`).


The descriptions in the prompts below describe in detail how they operate:

In [5]:
from utils import show_prompt

from prompts import (
    LS_DESCRIPTION,
    READ_FILE_DESCRIPTION,
    WRITE_FILE_DESCRIPTION,
)

show_prompt(LS_DESCRIPTION)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  List all files in the virtual filesystem stored in agent state.                                                │
│                                                                                                                 │
│  Shows what files currently exist in agent memory. Use this to orient yourself before other file operations     │
│  and maintain awareness of your file organization.                                                              │
│                                                                                                                 │
│  No parameters required - simply call ls() to see all available files.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
show_prompt(READ_FILE_DESCRIPTION)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Read content from a file in the virtual filesystem with optional pagination.                                   │
│                                                                                                                 │
│  This tool returns file content with line numbers (like `cat -n`) and supports reading large files in chunks    │
│  to avoid context overflow.                                                                                     │
│                                                                                                                 │
│  Parameters:                                                                                                    │
│  - file_path (required): Path to the file you want to read                                                      │
│  - offset (optional, default=0): Line number to start reading from                                              │
│  - limit (optional, default=2000): Maximum number of lines to read                                              │
│                                                                                                                 │
│  Essential before making any edits to understand existing content. Always read a file before editing it.        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
show_prompt(WRITE_FILE_DESCRIPTION)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Create a new file or completely overwrite an existing file in the virtual filesystem.                          │
│                                                                                                                 │
│  This tool creates new files or replaces entire file contents. Use for initial file creation or complete        │
│  rewrites. Files are stored persistently in agent state.                                                        │
│                                                                                                                 │
│  Parameters:                                                                                                    │
│  - file_path (required): Path where the file should be created/overwritten                                      │
│  - content (required): The complete content to write to the file                                                │
│                                                                                                                 │
│  Important: This replaces the entire file content.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Let's implement these functions below. There are two items worth noting. The first is the use of `@tool(description=PROMPT)`. Note that when `description="xyz" is in the tool decorator, "xyz" is sent to the LLM and the docstring is suppressed. It is often more convenient to have lengthy descriptions in a separate prompts file. This provides space to explain both the operation of the tool and how it should be used in this application. The second item is the error messages. These messages are intended for the LLM vs a human user. In an agentic system, the LLM can use the information in error messages to retry the operation. 

In [11]:
"""Virtual file system tools for agent state management.

This module provides tools for managing a virtual filesystem stored in agent state,
enabling context offloading and information persistence across agent interactions.
"""

from typing import Annotated

from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from langgraph.types import Command

from prompts import (
    LS_DESCRIPTION,
    READ_FILE_DESCRIPTION,
    WRITE_FILE_DESCRIPTION,
)
from deepagents import DeepAgentState


@tool(description=LS_DESCRIPTION)
def ls(state: Annotated[DeepAgentState, InjectedState]) -> list[str]:
    """List all files in the virtual filesystem."""
    return list(state.get("files", {}).keys())


@tool(description=READ_FILE_DESCRIPTION, parse_docstring=True)
def read_file(
    file_path: str,
    state: Annotated[DeepAgentState, InjectedState],
    offset: int = 0,
    limit: int = 2000,
) -> str:
    """Read file content from virtual filesystem with optional offset and limit.

    Args:
        file_path: Path to the file to read
        state: Agent state containing virtual filesystem (injected in tool node)
        offset: Line number to start reading from (default: 0)
        limit: Maximum number of lines to read (default: 2000)

    Returns:
        Formatted file content with line numbers, or error message if file not found
    """
    files = state.get("files", {})
    if file_path not in files:
        return f"Error: File '{file_path}' not found"

    content = files[file_path]
    if not content:
        return "System reminder: File exists but has empty contents"

    lines = content.splitlines()
    start_idx = offset
    end_idx = min(start_idx + limit, len(lines))

    if start_idx >= len(lines):
        return f"Error: Line offset {offset} exceeds file length ({len(lines)} lines)"

    result_lines = []
    for i in range(start_idx, end_idx):
        line_content = lines[i][:2000]  # Truncate long lines
        result_lines.append(f"{i + 1:6d}\t{line_content}")

    return "\n".join(result_lines)


@tool(description=WRITE_FILE_DESCRIPTION, parse_docstring=True)
def write_file(
    file_path: str,
    content: str,
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
) -> Command:
    """Write content to a file in the virtual filesystem.

    Args:
        file_path: Path where the file should be created/updated
        content: Content to write to the file
        state: Agent state containing virtual filesystem (injected in tool node)
        tool_call_id: Tool call identifier for message response (injected in tool node)

    Returns:
        Command to update agent state with new file content
    """
    files = state.get("files", {})
    files[file_path] = content
    return Command(
        update={
            "files": files,
            "messages": [
                ToolMessage(f"Updated file {file_path}", tool_call_id=tool_call_id)
            ],
        }
    )

# Revisiting state and reducer 
In the previous notebook, we defined the file state and reducer, but did not describe them. Let's do that here.
In `DeepAgentState`, `files` is defined as a dictionary with a key and value. As was mentioned above, the key is the filename and the value is the content of the file. `files` are added to state using the `file_reducer` when the `Command` in `write_files` is executed. In this reducer, `left` is the existing files in state and `right` is new values. The final statement allows the new values to overwrite old values: `{**left, **right}`, Python unpacks `left` first, then `right`. Any duplicate keys from `right` overwrite the earlier values from `left`.

```python
def file_reducer(left, right):
    """Merge two file dictionaries, with right side taking precedence.

    Used as a reducer function for the files field in agent state,
    allowing incremental updates to the virtual file system.

    Args:
        left: Left side dictionary (existing files)
        right: Right side dictionary (new/updated files)

    Returns:
        Merged dictionary with right values overriding left values
    """
    if left is None:
        return right
    elif right is None:
        return left
    else:
        return {**left, **right}


class DeepAgentState(AgentState):
    """Extended agent state that includes task tracking and virtual file system.

    Inherits from LangGraph's AgentState and adds:
    - todos: List of Todo items for task planning and progress tracking
    - files: Virtual file system stored as dict mapping filenames to content
    """

    todos: NotRequired[list[Todo]]
    files: Annotated[NotRequired[dict[str, str]], file_reducer]
```

We have a virtual filesystem and tools to work with it. Let's build a simple flight search agent to try it out. 
The agent will store the user's flight request and then search for available flights before returning structured JSON data with flight details!

Note that this simple approach is [very useful with long-running agent trajectories](https://www.anthropic.com/engineering/multi-agent-research-system#:~:text=Long%2Dhorizon%20conversation,across%20extended%20interactions.)! In this simple example, all information is easily kept in context, but for long-running agents, context content can be compressed or eliminated. Storing information prior to compression and retrieving it when it's needed is smart context engineering.

In [27]:
# File usage instructions
FILE_USAGE_INSTRUCTIONS = """You have access to a virtual file system to help you retain and save context.                                  
                                                                                                                
## Workflow Process                                                                                            
1. **Orient**: Use ls() to see existing files before starting work                                             
2. **Save**: Use write_file() to store the user's flight search request so that we can keep it for later                     
3. **Search**: Use search_flights() to find available flights based on the user's criteria
4. **Process**: Extract key flight details and prepare structured response
5. **Return**: Provide final response as JSON with required fields: origin, destination, departure_date, return_date"""

# Add flight search instructions
FLIGHT_SEARCH_INSTRUCTIONS = """IMPORTANT: You are a flight search agent that helps users find flights.

When processing flight requests:
1. Extract the origin, destination, departure_date, and return_date from user input
2. Use the search_flights tool to find available flights
3. Store the search results in a file for reference
4. Analyze the results to find the cheapest flight option
5. Return the flight details in JSON format with fields: airline, price, origin, destination, departure_date, return_date.
Always return valid JSON at the end of your response."""

# Full prompt
INSTRUCTIONS = (
    FILE_USAGE_INSTRUCTIONS + "\n\n" + "=" * 80 + "\n\n" + FLIGHT_SEARCH_INSTRUCTIONS
)
show_prompt(INSTRUCTIONS)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You have access to a virtual file system to help you retain and save context.                                  │
│                                                                                                                 │
│  ## Workflow Process                                                                                            │
│  1. **Orient**: Use ls() to see existing files before starting work                                             │
│  2. **Save**: Use write_file() to store the user's flight search request so that we can keep it for later       │
│  3. **Search**: Use search_flights() to find available flights based on the user's criteria                     │
│  4. **Process**: Extract key flight details and prepare structured response                                     │
│  5. **Return**: Provide final response as JSON with required fields: origin, destination, departure_date,       │
│  return_date                                                                                                    │
│                                                                                                                 │
│  ================================================================================                               │
│                                                                                                                 │
│  IMPORTANT: You are a flight search agent that helps users find flights.                                        │
│                                                                                                                 │
│  When processing flight requests:                                                                               │
│  1. Extract the origin, destination, departure_date, and return_date from user input                            │
│  2. Use the search_flights tool to find available flights                                                       │
│  3. Store the search results in a file for reference                                                            │
│  4. Analyze the results to find the cheapest flight option                                                      │
│  5. Return the flight details in JSON format with fields: airline, price, origin, destination, departure_date,  │
│  return_date.                                                                                                   │
│  Always return valid JSON at the end of your response.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [29]:
from IPython.display import Image, display
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from utils import format_messages
import json
import random
from datetime import datetime, timedelta

from deepagents.tools import ls, read_file, write_file
from deepagents import DeepAgentState
import os

# Mock flight search tool that returns realistic messy web scraping data
@tool(parse_docstring=True)
def search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    return_date: str = None
):
    """Search for flights between two airports on specified dates.

    This tool searches for available flights and returns flight options
    with details including airlines, times, prices, and aircraft information.

    Args:
        origin: Origin airport code (e.g., 'LHR', 'JFK', 'LAX')
        destination: Destination airport code (e.g., 'JFK', 'CDG', 'NRT')  
        departure_date: Departure date in YYYY-MM-DD format
        return_date: Return date in YYYY-MM-DD format (optional for one-way trips)

    Returns:
        Flight search results with available options and pricing.

    Example:
        search_flights("LHR", "JFK", "2025-03-15", "2025-03-22")
    """
    
    # Realistic messy flight search results like from a travel website
    messy_results = f"""
FLIGHT SEARCH RESULTS - FlightBooker.com
========================================

Search performed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Route: {origin} → {destination}
Outbound: {departure_date}
{"Return: " + return_date if return_date else "One-way ticket"}

Loading... ✓ Complete

BEST VALUE    SHORTEST    DIRECT FLIGHTS    CHEAPEST

Found 47 flights matching your criteria
Showing results 1-8 of 47

OUTBOUND FLIGHTS - {departure_date}
══════════════════════════════════

[SPONSORED] ✈ British Airways - Premium Partner
BA117 • DIRECT FLIGHT
Depart: London Heathrow (LHR) Terminal 5 - 08:30 AM
Arrive: New York JFK Terminal 7 - 11:45 AM (local time)
Flight time: 8h 15m | Aircraft: Boeing 777-300ER
Seat selection: From £35 | Meals included
Economy from £685 | Premium Economy from £1,245 | Business from £3,890
Baggage: 23kg included | Seat pitch: 31" | WiFi: £8.99
★★★★☆ 4.2/5 (2,847 reviews) "Comfortable seats, good service"

CHEAPEST PRICE ⚡
VS3 Virgin Atlantic • DIRECT
LHR T3 14:20 → JFK T4 17:30 (8h 10m)
Airbus A350-1000 | Seat 31A-F available
From £642 Economy Light (no changes, 1 bag)
£734 Economy Classic (changes allowed, seat selection)
£896 Premium Economy (priority boarding, extra legroom)
Inflight: Premium entertainment, USB charging
Baggage policy: Carry-on 10kg, Checked 23kg (Economy Classic+)
★★★★☆ 4.1/5 "Great value, modern aircraft"

American Airlines AA100 • ONEWORLD ALLIANCE
Depart LHR Terminal 3 - 10:15 AM
Arrive JFK Terminal 8 - 13:25 PM
Duration: 8 hours 10 minutes
Boeing 787-9 Dreamliner | 214 seats total
Main Cabin from $598 USD (approx £478)
Main Cabin Extra +£89 (extra legroom)
Business Class from $2,847 USD 
Admirals Club lounge access available
AAdvantage miles: Earn 3,458 miles
Amenities: Personal device entertainment, meal included
Seat map: 32A,32B,32C available | Power outlets at every seat

Lufthansa LH400 + LH402 • STAR ALLIANCE
⚠️ 1 STOP in Frankfurt (FRA) - 55min layover
LHR T2 16:45 → FRA T1 20:00 (1h 15m)
FRA T1 20:55 → JFK T1 07:30+1 (8h 35m)
Total journey: 9h 45m
Aircrafts: A320neo + A340-600
Economy Light from £545 (basic fare)
Economy Classic £634 (standard fare + seat selection)
Business Class £2,899 (lie-flat seats, lounge access)
Miles & More: Earn 4,299 award miles
Connecting flight risk: Short connection time
Ground transport FRA: Terminal shuttle required

ALTERNATIVE DATES (Save up to £127!)
±3 days: March 12-18: from £498
±7 days: March 8-22: from £445

{"RETURN FLIGHTS - " + return_date if return_date else ""}
{"══════════════════════════════════" if return_date else ""}

{'''JFK → LHR March 22, 2025

Virgin Atlantic VS4 DIRECT
JFK T4 22:00 → LHR T3 09:15+1 (7h 15m overnight)
A350-1000 | Upper Class suites available
From £634 Economy | £1,890 Upper Class
Red eye flight - blanket & pillow provided

British Airways BA112 DIRECT  
JFK T7 20:30 → LHR T5 07:40+1 (7h 10m)
Boeing 777-200ER | Club World available  
From £679 Economy | £3,245 Club World
Overnight service with meal and breakfast

American Airlines AA109
JFK T8 23:59 → LHR T3 11:25+1 (7h 26m)
787-9 | Flagship Business available
From $612 USD Main Cabin
Late departure - fewer crowds''' if return_date else ''}

TOTAL TRIP PRICING (Round-trip):
Cheapest combination: £1,276 (VS outbound + VS return)
Best value: £1,364 (BA outbound + BA return) - Most reliable
Premium option: £1,998 (All Premium Economy)

Filters applied: Non-stop preferred | Major airlines only
Price includes: Taxes, fees, and carrier charges
Booking fee: £12.99 per passenger
Payment options: Credit card, PayPal, Apple Pay, Bank transfer

⚠️ Prices may change. Book within 24h to secure rates.
✓ Free cancellation within 24h of booking
📞 24/7 customer service: +44 20 7946 0958

Additional fees may apply for:
- Seat selection (£12-89)  
- Extra baggage (£45-120)
- In-flight meals (£18-35)
- Priority boarding (£15-25)
- Travel insurance (recommended £28)

Terms: Prices shown in GBP. USD prices are estimates.
Currency fluctuation may affect final price.
All times shown in local timezone.

[Cookie Settings] [Privacy Policy] [Terms of Service]
Powered by FlightBooker.com © 2025
    """.strip()
    
    return messy_results

from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from dotenv import load_dotenv 
load_dotenv()

os.environ["AZURE_INFERENCE_ENDPOINT"]  = os.getenv("AZURE_ENDPOINT")
os.environ["AZURE_INFERENCE_CREDENTIAL"] = os.getenv("AZURE_CREDENTIAL")

# Create agent using create_react_agent directly
model = AzureAIChatCompletionsModel(
    credential=os.getenv("AZURE_CREDENTIAL"),
    endpoint=os.getenv("AZURE_ENDPOINT"),
    model="gpt-5-mini",
)

tools = [ls, read_file, write_file, search_flights]

# Create agent with system prompt
agent = create_react_agent(
    model, tools, prompt=INSTRUCTIONS, state_schema=DeepAgentState
)

Start the graph with no `files` in state and a user flight search request.

In [30]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": """I need to find flights from London Heathrow (LHR) to New York JFK for 
                departure on 2025-03-15 and return on 2025-03-22. Help me find the cheapest option and return it 
                as json with fields: airline, price, origin, destination, departure_date, return_date.
                """,
            }
        ],
        "files": {},
    }
)
format_messages(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I need to find flights from London Heathrow (LHR) to New York JFK for                                           │
│                 departure on 2025-03-15 and return on 2025-03-22. Help me find the cheapest option and return   │
│ it                                                                                                              │
│                 as json with fields: airline, price, origin, destination, departure_date, return_date.          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: ls                                                                                                │
│    Args: {}                                                                                                     │
│    ID: call_nos3OvN1oGsJn4uHUueG2PCL                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "/tmp/flight_search_request_LHR_JFK_2025-03-15_2025-03-22.txt",                                  │
│   "content": "User flight search request:\nOrigin: LHR (London Heathrow)\nDestination: JFK (New York            │
│ JFK)\nDeparture date: 2025-03-15\nReturn date: 2025-03-22\nRequest: Find the cheapest round-trip flight and     │
│ return details as JSON with fields: airline, price, origin, destination, departure_date, return_date."          │
│ }                                                                                                               │
│    ID: call_JCoNMZkji7lehsJzNz40FyJZ                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file /tmp/flight_search_request_LHR_JFK_2025-03-15_2025-03-22.txt                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: search_flights                                                                                    │
│    Args: {                                                                                                      │
│   "origin": "LHR",                                                                                              │
│   "destination": "JFK",                                                                                         │
│   "departure_date": "2025-03-15",                                                                               │
│   "return_date": "2025-03-22"                                                                                   │
│ }                                                                                                               │
│    ID: call_I13xNn4v43W7AfIQwYHh2kLE                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ FLIGHT SEARCH RESULTS - FlightBooker.com                                                                        │
│ ========================================                                                                        │
│                                                                                                                 │
│ Search performed on: 2025-10-15 01:41:47                                                                        │
│ Route: LHR → JFK                                                                                                │
│ Outbound: 2025-03-15                                                                                            │
│ Return: 2025-03-22                                                                                              │
│                                                                                                                 │
│ Loading... ✓ Complete                                                                                           │
│                                                                                                                 │
│ BEST VALUE    SHORTEST    DIRECT FLIGHTS    CHEAPEST                                                            │
│                                                                                                                 │
│ Found 47 flights matching your criteria                                                                         │
│ Showing results 1-8 of 47                                                                                       │
│                                                                                                                 │
│ OUTBOUND FLIGHTS - 2025-03-15                                                                                   │
│ ══════════════════════════════════                                                                              │
│                                                                                                                 │
│ [SPONSORED] ✈ British Airways - Premium Partner                                                                 │
│ BA117 • DIRECT FLIGHT                                                                                           │
│ Depart: London Heathrow (LHR) Terminal 5 - 08:30 AM                                                             │
│ Arrive: New York JFK Terminal 7 - 11:45 AM (local time)                                                         │
│ Flight time: 8h 15m | Aircraft: Boeing 777-300ER                                                                │
│ Seat selection: From £35 | Meals included                                                                       │
│ Economy from £685 | Premium Economy from £1,245 | Business from £3,890                                          │
│ Baggage: 23kg included | Seat pitch: 31" | WiFi: £8.99                                                          │
│ ★★★★☆ 4.2/5 (2,847 reviews) "Comfortable seats, good service"                                                   │
│                                                                                                                 │
│ CHEAPEST PRICE ⚡                                                                                               │
│ VS3 Virgin Atlantic • DIRECT                                                                                    │
│ LHR T3 14:20 → JFK T4 17:30 (8h 10m)                                                                            │
│ Airbus A350-1000 | Seat 31A-F available                                                                         │
│ From £642 Economy Light (no changes, 1 bag)                                                                     │
│ £734 Economy Classic (changes allowed, seat selection)  

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "/tmp/flight_search_results_LHR_JFK_2025-03-15_2025-03-22.txt",                                  │
│   "content": "FLIGHT SEARCH RESULTS - FlightBooker.com\n========================================\n\nSearch      │
│ performed on: 2025-10-15 01:41:47\nRoute: LHR                                                                   │
│ \u000212\u0016\u0010\u0016\u0016\u0016\u0016\u0016\u0016\u0016\u0016\u0016 → JFK\nOutbound: 2025-03-15\nReturn: │
│ 2025-03-22\n\nLoading... \u0016\u0016 Complete\n\nBEST VALUE    SHORTEST    DIRECT FLIGHTS    CHEAPEST\n\nFound │
│ 47 flights matching your criteria\nShowing results 1-8 of 47\n\nOUTBOUND FLIGHTS -                              │
│ 2025-03-15\n══════════════════════════════════\n\n[SPONSORED] \u0006 British Airways - Premium Partner\nBA117   │
│ \u0006 DIRECT FLIGHT\nDepart: London Heathrow (LHR) Terminal 5 - 08:30 AM\nArrive: New York JFK Terminal 7 -    │
│ 11:45 AM (local time)\nFlight time: 8h 15m | Aircraft: Boeing 777-300ER\nSeat selection: From \u001635 | Meals  │
│ included\nEconomy from \u001685 | Premium Economy from \u0011,245 | Business from \u0013,890\nBaggage: 23kg     │
│ included | Seat pitch: 31\" | WiFi: \u0018.99\n\u0010\u0010\u0010\u0010\u0010 4.2/5 (2,847 reviews)             │
│ \"Comfortable seats, good service\"\n\nCHEAPEST PRICE \u001f\u001a\nVS3 Virgin Atlantic \u0006 DIRECT\nLHR T3   │
│ 14:20 \u0002162\u0002\b\u0002\u0002\u0002 → JFK T4 17:30 (8h 10m)\nAirbus A350-1000 | Seat 31A-F                │
│ available\nFrom \u001642 Economy Light (no changes, 1 bag)\n\u0016734 Economy Classic (changes allowed, seat    │
│ selection)\n\u0016896 Premium Economy (priority boarding, extra legroom)\nInflight: Premium entertainment, USB  │
│ charging\nBaggage policy: Carry-on 10kg, Checked 23kg (Economy Classic+)\n\u0010\u0010\u0010\u0010 4.1/5        │
│ \"Great value, modern aircraft\"\n\nAmerican Airlines AA100 \u0006 ONEWORLD ALLIANCE\nDepart LHR Terminal 3 -   │
│ 10:15 AM\nArrive JFK Terminal 8 - 13:25 PM\nDuration: 8 hours 10 minutes\nBoeing 787-9 Dreamliner | 214 seats   │
│ total\nMain Cabin from $598 USD (approx \u0016378)\nMain Cabin Extra +\u001689 (extra legroom)\nBusiness Class  │
│ from $2,847 USD \nAdmirals Club lounge access available\nAmenities: Personal device entertainment, meal         │
│ included\nSeat map: 32A,32B,32C available | Power outlets at every seat\n\nLufthansa LH400 + LH402 \u0006 STAR  │
│ ALLIANCE\n\u0006 1 STOP in Frankfurt (FRA) - 55min layover\nLHR T2 16:45 \u0002\u0002 → FRA T1 20:00 (1h        │
│ 15m)\nFRA T1 20:55 \u0002\u0002 → JFK T1 07:30+1 (8h 35m)\nTotal journey: 9h 45m\nAircrafts: A320neo +          │
│ A340-600\nEconomy Light from \u001645 (basic fare)\nEconomy Classic \u001634 (standard fare + seat              │
│ selection)\nBusiness Class \u0012,899 (lie-flat seats, lounge access)\nMiles & More: Earn 4,299 award           │
│ miles\nConnecting flight risk: Short connection time\nGround transport FRA: Terminal shuttle                    │
│ required\n\nALTERNATIVE DATES (Save up to \u0016127!)\n\u0000\u0000\u0000\u0000\u0000 3 days: March 12-18: from │
│ \u0016498\n\u0000\u0000\u0000\u0000\u0000 7 days: March 8-22: from \u0016445\n\nRETURN FLIGHTS -                │
│ 2025-03-22\n══════════════════════════════════\n\nJFK \u0002\u0002→ LHR March 22, 2025\n\nVirgin Atlantic VS4   │
│ DIRECT\nJFK T4 22:00 \u0002\u0002→ LHR T3 09:15+1 (7h 15

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file /tmp/flight_search_results_LHR_JFK_2025-03-15_2025-03-22.txt                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ {"airline":"Virgin                                                                                              │
│ Atlantic","price":"£1,276","origin":"LHR","destination":"JFK","departure_date":"2025-03-15","return_date":"2025 │
│ -03-22"}                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

We can see the flight search request and results saved in our mock file system.

In [33]:
result["files"]

{'/tmp/flight_search_request_LHR_JFK_2025-03-15_2025-03-22.txt': 'User flight search request:\nOrigin: LHR (London Heathrow)\nDestination: JFK (New York JFK)\nDeparture date: 2025-03-15\nReturn date: 2025-03-22\nRequest: Find the cheapest round-trip flight and return details as JSON with fields: airline, price, origin, destination, departure_date, return_date.',
 '/tmp/flight_search_results_LHR_JFK_2025-03-15_2025-03-22.txt': 'FLIGHT SEARCH RESULTS - FlightBooker.com\n========================================\n\nSearch performed on: 2025-10-15 01:41:47\nRoute: LHR \x0212\x16\x10\x16\x16\x16\x16\x16\x16\x16\x16\x16 → JFK\nOutbound: 2025-03-15\nReturn: 2025-03-22\n\nLoading... \x16\x16 Complete\n\nBEST VALUE    SHORTEST    DIRECT FLIGHTS    CHEAPEST\n\nFound 47 flights matching your criteria\nShowing results 1-8 of 47\n\nOUTBOUND FLIGHTS - 2025-03-15\n══════════════════════════════════\n\n[SPONSORED] \x06 British Airways - Premium Partner\nBA117 \x06 DIRECT FLIGHT\nDepart: London Heathrow